# lib

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.5 MB/s eta 0:00:00


In [ ]:
pip install vncorenlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 46.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645952 sha256=aadf09a087fc30e9a66ee282245e591a8dec06859d0961a05a5766eaf2b68ccd
  Stored in directory: /root/.cache/pip/wheels/b2/1d/ea/9b94491d097561e07095d0800f2c5350ab183b49fc61780a8b
Successfully built vncorenlp


In [ ]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!mv VnCoreNLP-1.1.1.jar vncorenlp/

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/


!mkdir -p vncorenlp/models/postagger
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/postagger/vi-tagger 
!mv vi-tagger vncorenlp/models/postagger/


!mkdir -p vncorenlp/models/ner
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/ner/vi-500brownclusters.xz
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/ner/vi-ner.xz 
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/ner/vi-pretrainedembeddings.xz
!mv vi-500brownclusters.xz vncorenlp/models/ner/
!mv vi-ner.xz vncorenlp/models/ner/
!mv vi-pretrainedembeddings.xz vncorenlp/models/ner/

--2023-03-28 05:05:01--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   168MB/s    in 0.2s    

2023-03-28 05:05:03 (168 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2023-03-28 05:05:03--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting respon

In [ ]:
VNCORE_NLP = False
max_len = 50

In [ ]:
from vncorenlp import VnCoreNLP
vncorenlp = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg") 

ModuleNotFoundError: ignored

In [ ]:
def tokennize_vn(text):
    sentences = vncorenlp.tokenize(text)
    s = ''
    for t in sentences:
        s = s + ' '.join(t) + ' '
    return s

# Read data

In [ ]:
train_dataset = 'drive/MyDrive/CODE/ViTHSD/dataset/train.xlsx'
dev_dataset = 'drive/MyDrive/CODE/ViTHSD/dataset/dev.xlsx'
test_dataset = 'drive/MyDrive/CODE/ViTHSD/dataset/test.xlsx'

In [ ]:
import pandas as pd

train = pd.read_csv(train_dataset)
dev = pd.read_csv(dev_dataset)
test = pd.read_csv(test_dataset)

In [ ]:
len(train)

7000

In [ ]:
len(dev)

1201

In [ ]:
len(test)

1800

# Feature extraction  

## Make data

In [ ]:
import itertools

aspect = list(train.columns.values[2:])
sent = [1,2,3]

itertools.product([aspect, sent])

combined_label = [a for a in itertools.product(aspect, sent)]

In [ ]:
combined_label

[('individual', 1),
 ('individual', 2),
 ('individual', 3),
 ('groups', 1),
 ('groups', 2),
 ('groups', 3),
 ('religion/creed', 1),
 ('religion/creed', 2),
 ('religion/creed', 3),
 ('race/ethnicity', 1),
 ('race/ethnicity', 2),
 ('race/ethnicity', 3),
 ('politics', 1),
 ('politics', 2),
 ('politics', 3)]

In [ ]:
import numpy as np 
import pandas as pd

def make_data(data):
    content = []
    label = []

    for i in range(0, len(data)):
        row = data.iloc[i]
        if VNCORE_NLP:
            content.append(tokennize_vn(str(row['content'])))
        else:
            content.append(row['content'])
        default_lab = np.zeros(len(combined_label), dtype=int)
        for j in range(0, len(combined_label)):
            l = combined_label[j]
            if row[l[0]] == l[1]:
                default_lab[j] = 1
        label.append(default_lab)

    return pd.DataFrame({
        'Content': content,
        'Labels': label
    })

In [ ]:
train_data = make_data(train)
dev_data = make_data(dev)
test_data = make_data(test)

In [ ]:
test_data[1:10]

,Content,Labels
1,Đúng là ông bà có câu nhất lé quả không sai 🤔🤔...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,đéo hiểu sao 1 thằng đui và không có não mà lê...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,Hiếu Bùi sủa dơ,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,A vẫn chăm chỉ nayd a vẫn cố găng cho đến khi ...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,"Thái Lan hèn thật, người ta là đội ngũ tiên ph...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1]"
6,"Chau oi bn la, con cua, cô hả","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,Sao k có đoạn này ta,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
8,"Về chính trị, chúng tuyệt đối không cho nhân d...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
9,Gọi mấy thằng CA là phải dí thẳng tiền...,"[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


## Features

In [ ]:
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['Content'])
vocab_size = len(tokenizer.word_index) + 1

def encoding(X, y):
    X = tokenizer.texts_to_sequences(X.values.astype(str))
    X = pad_sequences(X, maxlen=max_len, padding='post')
    y = np.asarray([np.asarray(row, dtype=float) for row in y], dtype=int)
    return (X,y)

In [ ]:
# import pickle

# # saving
# with open('drive/MyDrive/CODE/ViTHSD/model/tokenizer1.pickle', 'wb') as handle:
#     pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
train_features = encoding(train_data['Content'], train_data['Labels'])
dev_features = encoding(dev_data['Content'], dev_data['Labels'])
test_features = encoding(test_data['Content'], test_data['Labels'])

## Evaluation metric

In [ ]:
# Evaluation metric
import sys
import os
import os.path
from scipy.stats import sem
import numpy as np
from ast import literal_eval
import tensorflow as tf

def em_score(y_true, y_pred):
    MR = np.all(y_pred == y_true, axis=1).mean()
    return MR

def accuracy_score(y_true, y_pred):
    temp = 0
    for i in range(0, len(y_true)):
        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    return temp / len(y_true)

 
def f1_score(y_true, y_pred):
    temp = 0
    for i in range(len(y_true)):
        if (sum(y_true[i]) == 0) and (sum(y_pred[i]) == 0):
            continue
        temp+= (2*sum(np.logical_and(y_true[i], y_pred[i]))) / (sum(y_true[i])+sum(y_pred[i]))
    return temp/ len(y_true)

# Model

## Pre-trained Embedding 

In [ ]:
import numpy as np

EMBEDING_DIM = 300
NUM_LABEL = 15
EMBEDDING = 'drive/My Drive/CODE/ViTHSD/cc.vi.300.vec'
MAX_FEATURE = 10000

embeddings_index = dict()
f = open(EMBEDDING)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


embedding_matrix = np.zeros((vocab_size, EMBEDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Loaded 2000000 word vectors.


## Train 

In [ ]:
# Bi-GRU-LSTM-CNN

import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential, Model

from keras.layers import Dense, Concatenate, Bidirectional, GRU, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, SpatialDropout1D, Dropout, concatenate
from keras.layers import Flatten, LSTM, Input
from tensorflow.keras.layers import Embedding

units = 100

input = Input(shape = (max_len,))
emb = Embedding(vocab_size, EMBEDING_DIM, weights=[embedding_matrix], input_length=max_len, trainable=True)(input)
x1 = SpatialDropout1D(0.2)(emb)

x = Bidirectional(GRU(units, return_sequences = True))(x1)
x = Conv1D(int(units/2), kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    
y = Bidirectional(LSTM(units, return_sequences = True))(x1)
y = Conv1D(int(units/2), kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(y)
    
avg_pool1 = GlobalAveragePooling1D()(x)
max_pool1 = GlobalMaxPooling1D()(x)
    
avg_pool2 = GlobalAveragePooling1D()(y)
max_pool2 = GlobalMaxPooling1D()(y)
    
    
x = Concatenate(axis=-1)([avg_pool1, max_pool1, avg_pool2, max_pool2])
out = Dense(15, activation = "sigmoid")(x)

model = Model(input, out)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 300)     2132400     ['input_1[0][0]']                
                                                                                                  
 spatial_dropout1d (SpatialDrop  (None, 100, 300)    0           ['embedding[0][0]']              
 out1D)                                                                                           
                                                                                                  
 bidirectional (Bidirectional)  (None, 100, 200)     241200      ['spatial_dropout1d[0][0]']  

In [ ]:
model.fit(train_features[0].astype(np.float32), train_features[1], validation_data=(dev_features[0].astype(np.float32), dev_features[1]), 
          batch_size=128, epochs=30, verbose=True)

Epoch 1/30
55/55 [==============================] - 22s 199ms/step - loss: 0.3012 - acc: 0.4026 - val_loss: 0.2164 - val_acc: 0.5679
Epoch 2/30
55/55 [==============================] - 7s 132ms/step - loss: 0.2034 - acc: 0.5904 - val_loss: 0.1899 - val_acc: 0.6020
Epoch 3/30
55/55 [==============================] - 6s 105ms/step - loss: 0.1777 - acc: 0.6424 - val_loss: 0.1809 - val_acc: 0.6103
Epoch 4/30
55/55 [==============================] - 6s 105ms/step - loss: 0.1577 - acc: 0.6627 - val_loss: 0.1823 - val_acc: 0.6087
Epoch 5/30
55/55 [==============================] - 4s 78ms/step - loss: 0.1406 - acc: 0.6849 - val_loss: 0.1862 - val_acc: 0.5928
Epoch 6/30
55/55 [==============================] - 4s 66ms/step - loss: 0.1248 - acc: 0.6956 - val_loss: 0.1938 - val_acc: 0.5321
Epoch 7/30
55/55 [==============================] - 4s 69ms/step - loss: 0.1098 - acc: 0.7221 - val_loss: 0.2170 - val_acc: 0.5828
Epoch 8/30
55/55 [==============================] - 3s 50ms/step - loss: 0.096

## Eval

In [ ]:
y_test_pred = model.predict(test_features[0])

57/57 [==============================] - 2s 9ms/step


In [ ]:
y_test_pred_new = []

for y in y_test_pred:
    lb = []
    for i in range(0, len(y)):
        if y[i] >= 0.5:
            lb.append(1)
        else:
            lb.append(0)
    y_test_pred_new.append(lb)

In [ ]:
f1_score(test_features[1], y_test_pred_new)*100

39.910052910052826

# Model - pre-trained transformers

## Pre-trained Embedding 

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("NlpHUST/vibert4news-base-cased")
embedding_matrix = model.embeddings.word_embeddings.weight.detach().numpy()
# dim = model.embeddings.word_embeddings.embedding_dim
# vocab = model.embeddings.word_embeddings.num_embeddings
dim = embedding_matrix.shape[1]
vocab = embedding_matrix.shape[0]

Some weights of the model checkpoint at NlpHUST/vibert4news-base-cased were not used when initializing IBertModel: ['bert.encoder.layer.3.attention.output.dense.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.10.attention.self.query.weight', 'bert.encoder.layer.5.attention.self.key.weight', 'bert.encoder.layer.7.attention.self.key.bias', 'bert.encoder.layer.5.intermediate.dense.weight', 'bert.encoder.layer.6.attention.output.LayerNorm.bias', 'bert.encoder.layer.5.attention.self.value.weight', 'bert.encoder.layer.8.attention.self.query.bias', 'bert.encoder.layer.9.attention.output.dense.weight', 'bert.encoder.layer.1.attention.self.key.weight', 'bert.encoder.layer.2.attention.self.value.bias', 'bert.encoder.layer.6.output.dense.bias', 'bert.encoder.layer.6.attention.self.value.weight', 'bert.encoder.layer.9.attention.output.LayerNorm.bias', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.8.attention.self.value.bias', 'bert.encoder.layer.6.atten

## Train 

In [ ]:
# Bi-GRU-LSTM-CNN

import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential, Model

from keras.layers import Dense, Concatenate, Bidirectional, GRU, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, SpatialDropout1D, Dropout, concatenate
from keras.layers import Flatten, LSTM, Input
from tensorflow.keras.layers import Embedding

units = 100

input = Input(shape = (max_len,))
emb = Embedding(vocab, dim, weights=[embedding_matrix], input_length=max_len, trainable=True)(input)
x1 = SpatialDropout1D(0.2)(emb)

x = Bidirectional(GRU(units, return_sequences = True))(x1)
x = Conv1D(int(units/2), kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    
y = Bidirectional(LSTM(units, return_sequences = True))(x1)
y = Conv1D(int(units/2), kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(y)
    
avg_pool1 = GlobalAveragePooling1D()(x)
max_pool1 = GlobalMaxPooling1D()(x)
    
avg_pool2 = GlobalAveragePooling1D()(y)
max_pool2 = GlobalMaxPooling1D()(y)
    
    
x = Concatenate(axis=-1)([avg_pool1, max_pool1, avg_pool2, max_pool2])
out = Dense(15, activation = "sigmoid")(x)

model = Model(input, out)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 50, 768)      47616000    ['input_5[0][0]']                
                                                                                                  
 spatial_dropout1d_3 (SpatialDr  (None, 50, 768)     0           ['embedding_3[0][0]']            
 opout1D)                                                                                         
                                                                                                  
 bidirectional_4 (Bidirectional  (None, 50, 200)     522000      ['spatial_dropout1d_3[0][0]

In [ ]:
model.fit(train_features[0].astype(np.float32), train_features[1], validation_data=(dev_features[0].astype(np.float32), dev_features[1]), 
          batch_size=128, epochs=3, verbose=True)

Epoch 1/3
55/55 [==============================] - 14s 130ms/step - loss: 0.2739 - acc: 0.4164 - val_loss: 0.2147 - val_acc: 0.5512
Epoch 2/3
55/55 [==============================] - 5s 87ms/step - loss: 0.1999 - acc: 0.6029 - val_loss: 0.1973 - val_acc: 0.5912
Epoch 3/3
55/55 [==============================] - 4s 75ms/step - loss: 0.1749 - acc: 0.6489 - val_loss: 0.1908 - val_acc: 0.6037


## Eval

In [ ]:
y_test_pred = model.predict(test_features[0])

57/57 [==============================] - 1s 5ms/step


In [ ]:
y_test_pred_new = []

for y in y_test_pred:
    lb = []
    for i in range(0, len(y)):
        if y[i] >= 0.5:
            lb.append(1)
        else:
            lb.append(0)
    y_test_pred_new.append(lb)

In [ ]:
f1_score(test_features[1], y_test_pred_new)*100

36.09087301587298

In [ ]:
# model.save("drive/MyDrive/CODE/ViTHSD/model/model1.h5")

# Model - pre-trained transformers 2

## Pre-trained Embedding 

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("bert-base-multilingual-cased")
embedding_matrix = model.embeddings.word_embeddings.weight.detach().numpy()
dim = model.embeddings.word_embeddings.embedding_dim
vocab = model.embeddings.word_embeddings.num_embeddings
# dim = embedding_matrix.shape[1]
# vocab = embedding_matrix.shape[0]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Train 

In [ ]:
# TextCNN

import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential, Model

from keras.layers import Dense, Concatenate, Bidirectional, GRU, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, SpatialDropout1D, Dropout, concatenate
from keras.layers import Flatten, LSTM, Input, Reshape, Conv2D, MaxPool2D
from tensorflow.keras.layers import Embedding

filter_sizes = [1,2,3,5]
num_filters = 32

input = Input(shape=(max_len,))
x = Embedding(vocab, dim, weights=[embedding_matrix], trainable = False)(input)
x = SpatialDropout1D(0.4)(x)
x = Reshape((max_len, dim, 1))(x)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], dim), kernel_initializer='normal',
                activation='elu')(x)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], dim), kernel_initializer='normal',
                activation='elu')(x)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], dim), kernel_initializer='normal',
                activation='elu')(x)
conv_3 = Conv2D(num_filters, kernel_size=(filter_sizes[3], dim), kernel_initializer='normal',
                activation='elu')(x)

maxpool_0 = MaxPool2D(pool_size=(max_len - filter_sizes[0] + 1, 1))(conv_0)
maxpool_1 = MaxPool2D(pool_size=(max_len - filter_sizes[1] + 1, 1))(conv_1)
maxpool_2 = MaxPool2D(pool_size=(max_len - filter_sizes[2] + 1, 1))(conv_2)
maxpool_3 = MaxPool2D(pool_size=(max_len - filter_sizes[3] + 1, 1))(conv_3)

z = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3])
z = Flatten()(z)
z = Dropout(0.1)(z)

out = Dense(15, activation = "sigmoid")(z)

model = Model(input, out)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 100, 768)     91812096    ['input_3[0][0]']                
                                                                                                  
 spatial_dropout1d_1 (SpatialDr  (None, 100, 768)    0           ['embedding_1[0][0]']            
 opout1D)                                                                                         
                                                                                                  
 reshape (Reshape)              (None, 100, 768, 1)  0           ['spatial_dropout1d_1[0][0]

In [ ]:
model.fit(train_features[0].astype(np.float32), train_features[1], validation_data=(dev_features[0].astype(np.float32), dev_features[1]), 
          batch_size=128, epochs=20, verbose=True)

Epoch 1/20
55/55 [==============================] - 3s 11ms/step - loss: 0.3270 - acc: 0.3596 - val_loss: 0.2278 - val_acc: 0.5154
Epoch 2/20
55/55 [==============================] - 0s 6ms/step - loss: 0.2333 - acc: 0.4566 - val_loss: 0.2235 - val_acc: 0.5346
Epoch 3/20
55/55 [==============================] - 0s 6ms/step - loss: 0.2299 - acc: 0.4843 - val_loss: 0.2210 - val_acc: 0.5429
Epoch 4/20
55/55 [==============================] - 0s 6ms/step - loss: 0.2260 - acc: 0.5004 - val_loss: 0.2185 - val_acc: 0.5437
Epoch 5/20
55/55 [==============================] - 0s 6ms/step - loss: 0.2216 - acc: 0.5150 - val_loss: 0.2160 - val_acc: 0.5479
Epoch 6/20
55/55 [==============================] - 0s 6ms/step - loss: 0.2175 - acc: 0.5299 - val_loss: 0.2135 - val_acc: 0.5562
Epoch 7/20
55/55 [==============================] - 0s 6ms/step - loss: 0.2129 - acc: 0.5451 - val_loss: 0.2109 - val_acc: 0.5604
Epoch 8/20
55/55 [==============================] - 0s 6ms/step - loss: 0.2095 - acc: 0.5

## Eval

In [ ]:
y_test_pred = model.predict(test_features[0])

57/57 [==============================] - 0s 2ms/step


In [ ]:
y_test_pred_new = []

for y in y_test_pred:
    lb = []
    for i in range(0, len(y)):
        if y[i] >= 0.5:
            lb.append(1)
        else:
            lb.append(0)
    y_test_pred_new.append(lb)

In [ ]:
f1_score(test_features[1], y_test_pred_new)*100

25.778042328042407